## Exploration of the data and graphs
### Installing requirements

In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 418 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 5.5 MB/s  eta 0:00:01
     |████████████████████████████████| 41.6 MB 6.0 MB/s eta 0:00:011    |███████▊                        | 10.0 MB 2.9 MB/s eta 0:00:11
     |████████████████████████████████| 26.7 MB 10.4 MB/s eta 0:00:01   |█████▊                          | 4.8 MB 9.3 MB/s eta 0:00:03     |██████████████████████████▍     | 22.0 MB 10.9 MB/s eta 0:00:01     |█████████████████████████████▎  | 24.5 MB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 6.3 MB/s eta 0:00:01
ERROR: streamlit-folium 0.6.4 has requirement streamlit>=1.2, but you'll have streamlit 1.0.0 which is incompatible.


### Imports

In [1]:
import pandas as pd
import plotly.express as px

### Pulling and preprocessing dataset

In [48]:
df = pd.read_excel('https://ansperformance.eu/download/xls/ASMA_Additional_Time.xlsx',sheet_name='DATA')
df['DATE'] = ('1/' + df['MONTH_MON'].apply(str) + '/' + df['YEAR'].apply(str)).apply(lambda l:datetime.strptime(l, '%d/%b/%Y'))
df['AVG_UNIMPEDED_ASMA_TIME'] = df['TIME_ASMA_UNIMP_2']/df['FLT_ASMA_UNIMP_2']
df['AVG_ADDITIONAL_ASMA_TIME'] = df['TIME_ASMA_ADD_2']/df['FLT_ASMA_UNIMP_2']
df.to_feather('resources/ASMA_Additional_Time.feather')

### Graphing airport authority

In [49]:
df = pd.read_feather('resources/ASMA_Additional_Time.feather')

In [46]:
APT_NAME = 'Brussels'
df_tmp = df[df['APT_NAME'] == APT_NAME]
df_tmp.rename({
    'YEAR':'Year',
    'MONTH_MON':'Month',
    'CHANGE_YEAR':'Period',
    'FLT_ASMA_UNIMP_2':'IFR flights with unimpeded reference time',
    'AVG_UNIMPEDED_ASMA_TIME':'Avg. unimpeded ASMA time',
    'AVG_ADDITIONAL_ASMA_TIME':'Avg. additional ASMA time',
    'CHANGE_AVG_UNIMPEDED_ASMA_TIME':'Pct. change in avg. unimpeded ASMA time',
    'CHANGE_AVG_ADDITIONAL_ASMA_TIME':'Pct. change in avg. additional ASMA time',
    }, axis=1, inplace=True)
fig = px.line(df_tmp, x='Month', y='IFR flights with unimpeded reference time', title='ASMA Additional Time for ' + APT_NAME,color='YEAR')
fig 

In [124]:
APT_NAME = 'Brussels'
df_tmp = df[df['APT_NAME'] == APT_NAME]
df_tmp = df_tmp.groupby('YEAR').mean()[['AVG_UNIMPEDED_ASMA_TIME','AVG_ADDITIONAL_ASMA_TIME']].reset_index()
df_tmp.rename({
    'YEAR':'Year',
    'CHANGE_YEAR':'Period',
    'AVG_UNIMPEDED_ASMA_TIME':'Avg. unimpeded ASMA time',
    'AVG_ADDITIONAL_ASMA_TIME':'Avg. additional ASMA time',
    'CHANGE_AVG_UNIMPEDED_ASMA_TIME':'Pct. change in avg. unimpeded ASMA time',
    'CHANGE_AVG_ADDITIONAL_ASMA_TIME':'Pct. change in avg. additional ASMA time',
    }, axis=1, inplace=True)
fig = px.bar(df_tmp, x='Year', y=['Avg. unimpeded ASMA time','Avg. additional ASMA time'], title='Average ASMA Time for ' + APT_NAME  )
fig.layout = dict(yaxis=dict(title='Avg. ASMA Time'))
fig

In [120]:
APT_NAME = 'Brussels'
df_tmp = df[df['APT_NAME'] == APT_NAME]
df_tmp = df_tmp.groupby('YEAR').mean()[['AVG_UNIMPEDED_ASMA_TIME','AVG_ADDITIONAL_ASMA_TIME']].reset_index()

df_tmp['CHANGE_AVG_UNIMPEDED_ASMA_TIME'] = df_tmp['AVG_UNIMPEDED_ASMA_TIME'].pct_change()*100
df_tmp['CHANGE_AVG_ADDITIONAL_ASMA_TIME'] = df_tmp['AVG_ADDITIONAL_ASMA_TIME'].pct_change()*100

In [121]:
df_tmp['CHANGE_YEAR'] = df_tmp['YEAR'].apply(lambda l: f"{l-1}-{l}")

In [122]:
df_tmp.rename({
    'CHANGE_YEAR':'Period',
    'AVG_UNIMPEDED_ASMA_TIME':'Avg. unimpeded ASMA time',
    'AVG_ADDITIONAL_ASMA_TIME':'Avg. additional ASMA time',
    'CHANGE_AVG_UNIMPEDED_ASMA_TIME':'Pct. change in avg. unimpeded ASMA time',
    'CHANGE_AVG_ADDITIONAL_ASMA_TIME':'Pct. change in avg. additional ASMA time',
    }, axis=1, inplace=True)
df_tmp = df_tmp[~df_tmp['Pct. change in avg. unimpeded ASMA time'].isna()]
fig = px.bar(df_tmp, x='Period', y=['Pct. change in avg. unimpeded ASMA time','Pct. change in avg. additional ASMA time'], title='Percentage change of average ASMA Time per year for ' + APT_NAME  )

In [123]:
fig